In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint

# Path to the folder containing separate class folders (jute, maize, rice, sugarcane, wheat)
train_data_dir = 'C:\\Users\\arusa\\Downloads\\archive (9)\\crop_images'
img_width, img_height = 224, 224  # Adjust based on the VGG16 input size

# Preprocess and augment the images for better training
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

# Set batch size
batch_size = 100

# Training data generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Load VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Create a new model on top
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))  # Change '5' to the number of classes you have

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=1)

# Saving the trained model
model.save('paddy_or_not.h5')

# Test the model on a single image to identify if it's rix ce or not
def test_model(test_image_path):
    img = image.load_img(test_image_path, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    result = model.predict(img)
    class_indices = {0: 'jute', 1: 'maize', 2: 'rice', 3: 'sugarcane', 4: 'wheat'}
    predicted_class = class_indices[np.argmax(result)]
    if predicted_class == 'rice':
        print("Yes")  # If the model predicts the image as rice
    else:
        print("Not rice")

# Replace 'path_to_your_test_image.jpg' with the path to your test image
test_image_path = 'C:\\Users\\arusa\\Downloads\\archive (9)\\test_crop_image\\rice-828540_1280.jpg'
test_model(test_image_path)  # Run the function for testing the model with a single image
